## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## loading all the evironment variables

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
## import the LLM model
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018CD9A3C220>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018CD9A3CEE0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is Aditya and I am a software engineer")])

## invoke -> Transform a single input into an output. Override to implement.

AIMessage(content="Hi Aditya, it's nice to meet you!\n\nIt's great to know you're a software engineer. What kind of software do you specialize in? Are you working on any interesting projects right now? 😊  \n\nI'm always eager to learn more about what people do in the tech world.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 21, 'total_tokens': 88, 'completion_time': 0.121818182, 'prompt_time': 0.000158929, 'queue_time': 0.01344923, 'total_time': 0.121977111}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-894a020a-bb80-489e-b6e3-873e5e5ff600-0', usage_metadata={'input_tokens': 21, 'output_tokens': 67, 'total_tokens': 88})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, my name is Aditya and I am a software engineer"),
        AIMessage(content="Hello Aditya, it's nice to meet you!\n\nI'm glad to know you're a software engineer. What kind of software do you specialize in? Do you have any interesting projects you're working on right now?  \n\nI'm always eager to learn more about what people do in the tech world. 😊  \n\n"),
        HumanMessage(content="Hey, What my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Aditya and that you are a software engineer!  \n\nIs there anything else you'd like to tell me about yourself or would you like to ask me something? 😄  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 112, 'total_tokens': 155, 'completion_time': 0.078181818, 'prompt_time': 0.003884908, 'queue_time': 0.013550401, 'total_time': 0.082066726}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5213aaaa-6e63-4d54-827a-f212c0310894-0', usage_metadata={'input_tokens': 112, 'output_tokens': 43, 'total_tokens': 155})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [ ]:
!pip install langchain_community

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory ## for chain

# dictonary
store={}

##    func name         parameter          return type
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]


## when we call this "get_session_history()" function it checks the session_id in the store={} dictonary, if its present the we are going to get the entire history from chatMessageHistory() function and then we are going to return it.


with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [7]:
config = {"configurable":{"session_id":"chat1"}}

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Aditya and I am a software engineer")],
    config=config
)

In [9]:
response.content

"It's nice to meet you, Aditya!\n\nAs a software engineer, what kind of projects are you working on these days?  \n\nI'm always interested in learning more about what people are building in the software world.\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config
)

AIMessage(content='Your name is Aditya. 😊  I remember that from our introduction!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 83, 'total_tokens': 100, 'completion_time': 0.030909091, 'prompt_time': 0.002332742, 'queue_time': 0.011853828, 'total_time': 0.033241833}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1244caa1-5c45-44a5-b8bb-d40b4a4cd9aa-0', usage_metadata={'input_tokens': 83, 'output_tokens': 17, 'total_tokens': 100})

In [11]:
## change the config i.e changing the session id

config1 = {"configurable":{"session_id":"chat2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)

response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="My name is Priyanshu")],
    config=config1
)

response.content

"It's nice to meet you, Priyanshu!  \n\nHow can I help you today? 😊  \n"

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)

response.content

'Your name is Priyanshu.  \n\nI remember you telling me earlier! 😄  Is there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [ ]:

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Hey, you are a helpful assistant. Answer all the question to best of you ability in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [17]:
chain.invoke({"messages": [HumanMessage(content="Hi, my name is Aditya")]})

AIMessage(content="Hello Aditya, it's nice to meet you!\n\nI'm ready to answer your questions to the best of my ability.  Just ask away! 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 32, 'total_tokens': 70, 'completion_time': 0.069090909, 'prompt_time': 0.001616265, 'queue_time': 0.015565182999999998, 'total_time': 0.070707174}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-89782359-e998-4d2c-9bcd-6901940d6dd5-0', usage_metadata={'input_tokens': 32, 'output_tokens': 38, 'total_tokens': 70})

In [18]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [19]:
config = {"configurable":{"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Aditya")],
    config=config
)

response

AIMessage(content="Hello Aditya, it's nice to meet you!\n\nI'm ready to answer your questions to the best of my ability. Just ask away! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 32, 'total_tokens': 68, 'completion_time': 0.065454545, 'prompt_time': 0.000699077, 'queue_time': -0.000655027, 'total_time': 0.066153622}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-34fd874d-03b2-4c8e-af27-1fd932b24c65-0', usage_metadata={'input_tokens': 32, 'output_tokens': 36, 'total_tokens': 68})

In [20]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [21]:
response = chain.invoke({"messages":[HumanMessage(content="Hi, My name is Aditya.")], "language":"Hindi"})

response.content

'नमस्ते आदित्य, मुझे खुशी है कि आप मुझसे जुड़े हैं! 😊 \n\nआप मुझसे क्या पूछना चाहेंगे? मैं आपकी सहायता करने के लिए तैयार हूँ।  \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [22]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [28]:
config = {"configurable":{"session_id": "chat4"}}

response = with_message_history.invoke(
    {
        'messages': [HumanMessage(content="Hi, I am aditya")],
        'language': "bhojpuri"
    },
    config = config
)
response.content

'नमस्ते आदित्य!  आपका स्वागत है! 😊 कबूतर के बारे में बताओ?  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [33]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 70,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [34]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages":messages+[HumanMessage(content="What Icecream do I like?")],
        "language": "English"
    }
)

response.content

'You said you like vanilla ice cream! 😊  '

In [35]:
response = chain.invoke(
    {
        "messages":messages+[HumanMessage(content="What math question i asked?")],
        "language": "English"
    }
)

response.content

'You asked "what is 2 + 2". 🍦 😄  \n\nAnything else I can help you with?\n'

In [36]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}